In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

LOGS_PATH = "../../image2sdf/logs/decoder/log.pkl"


In [ ]:
logs = pickle.load(open(LOGS_PATH, 'rb'))

In [ ]:
logs.keys()

In [ ]:
len(logs["sdf"])

In [ ]:
avrg_sdf = []
for i in range(len(logs["sdf"]) - 246):
    if i%1000 == 0:
        print(i)
    avrg_sdf.append(np.mean(logs["sdf"][i : i + 246]))

In [ ]:
print(np.mean(logs["sdf"][-246:]))

In [ ]:
import numpy as np
import torch
import pickle
import glob
import json
import time

from networks import Decoder
from dataLoader import DatasetDecoder
from marching_cubes_rgb import *

import IPython

# directory which contain the SDF input and where all the output will be generated
MAIN_DIR = "../../image2sdf/"
# MAIN_DIR = "/home/loic/MasterPDM/image2sdf/"

DECODER_PATH = "models_and_codes/decoderSDF.pth"
LATENT_CODE_PATH = "models_and_codes/latent_code.pkl"
LOGS_PATH = "../../image2sdf/logs/log.pkl"
PARAM_FILE = "config/param.json"

SDF_DIR = MAIN_DIR + "sdf/"
RESOLUTION = 64



def init_xyz(resolution):
    """ fill 3d grid representing 3d location to give as input to the decoder """
    xyz = torch.empty(resolution * resolution * resolution, 3).cuda()

    for x in range(resolution):
        for y in range(resolution):
            for z in range(resolution):
                xyz[x * resolution * resolution + y * resolution + z, :] = torch.Tensor([x/(resolution-1)-0.5,y/(resolution-1)-0.5,z/(resolution-1)-0.5])

    return xyz

def init_lat_vecs(num_scenes, latent_size):
    """initialize random latent code for every model"""

    lat_code_mu = torch.nn.Embedding(num_scenes, latent_size).cuda()
    torch.nn.init.normal_(
        lat_code_mu.weight.data,
        0.0,
        1.0,
    )
    lat_code_log_std = torch.nn.Embedding(num_scenes, latent_size).cuda()
    torch.nn.init.normal_(
        lat_code_log_std.weight.data,
        0.0,
        0.0,
    )

    return lat_code_mu, lat_code_log_std

def init_opt_sched(decoder, lat_vecs_mu, lat_vecs_log_std, param):
    """ initialize optimizer and scheduler"""

    optimizer = torch.optim.Adam(
        [
            {
                "params": decoder.parameters(),
                "lr": param["eta_decoder"],
            },
            {
                "params": lat_vecs_mu.parameters(),
                "lr": param["eta_latent_space_mu"],
            },
            {
                "params": lat_vecs_log_std.parameters(),
                "lr": param["eta_latent_space_std"],
            },
        ]
    )

    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=param["gammaLR"])

    return optimizer, scheduler

def compute_time_left(time_start, model_count, num_model, epoch, num_epoch):
    """ Compute time left until the end of training """
    time_passed = time.time() - time_start
    num_model_seen = epoch * num_model + model_count
    time_per_model = time_passed/num_model_seen
    estimate_total_time = time_per_model * num_epoch * num_model
    estimate_time_left = estimate_total_time - time_passed

    return estimate_time_left

def compute_loss(pred_sdf, pred_rgb, sdf_gt, rgb_gt, threshold_precision, param):
    """ compute sdf, rgb and regression loss """

    loss = torch.nn.MSELoss(reduction='none')

    # assign weight of 0 for easy samples that are well trained
    weight_sdf = ~((pred_sdf > threshold_precision).squeeze() * (sdf_gt > threshold_precision).squeeze()) \
        * ~((pred_sdf < -threshold_precision).squeeze() * (sdf_gt < -threshold_precision).squeeze())

    #L2 loss, only for hard samples
    loss_sdf = loss(pred_sdf.squeeze(), sdf_gt)
    loss_sdf = (loss_sdf * weight_sdf).sum()/weight_sdf.count_nonzero()
    loss_sdf *= param["lambda_sdf"]

    # loss rgb
    loss_rgb = loss(pred_rgb, rgb_gt)
    loss_rgb = ((loss_rgb[:,0] * weight_sdf) + (loss_rgb[:,1] * weight_sdf) + (loss_rgb[:,2] * weight_sdf)).sum()/weight_sdf.count_nonzero()
    loss_rgb *= param["lambda_rgb"]
    
    # regularization loss
    loss_kl = (-0.5 * (1 + lat_code_log_std.weight - lat_code_mu.weight.pow(2) - lat_code_log_std.weight.exp())).mean()
    loss_kl *= param["lambda_kl"]

    return loss_sdf, loss_rgb, loss_kl

if __name__ == '__main__':
    print("Loading parameters...")

    # load parameters
    param_all = json.load(open(PARAM_FILE))
    param = param_all["decoder"]

    resolution = RESOLUTION
    threshold_precision = 1.0/resolution
    # num_samples_per_model = resolution * resolution * resolution
    num_samples_per_model = param["num_samples_per_model"]
    batch_size = param["dataLoader"]["batch_size"]

    # get models' hashs
    list_model_hash = []
    for val in glob.glob(SDF_DIR + "*.h5"):
        list_model_hash.append(os.path.basename(val).split('.')[0])
    num_model = len(list_model_hash)

    # fill a xyz grid to give as input to the decoder 
    xyz = init_xyz(resolution)

    # initialize a random latent code for each models
    lat_code_mu, lat_code_log_std = init_lat_vecs(num_model, param["latent_size"])

    # create a dictionary going from an hash to a corresponding index
    idx = torch.arange(num_model).type(torch.LongTensor).cuda()
    dict_model_hash_2_idx = dict()
    for model_hash, i in zip(list_model_hash, range(num_model)):
        dict_model_hash_2_idx[model_hash] = idx[i]

In [ ]:
code_mu

In [ ]:
sdf_gt.reshape(10000)

In [ ]:
IPython.embed()

In [ ]:

loss_sdf, loss_rgb, loss_kl = compute_loss(pred_sdf, pred_rgb, sdf_gt.reshape(10000), rgb_gt.reshape(10000,3), threshold_precision, param)

In [ ]:
a = torch.tensor([[1,2,2],[3,4,3]])

In [ ]:
a[0]

In [ ]:
rgb_gt[:2,100:102]

In [ ]:
rgb_gt[:2,100:102].reshape(4,3)